## Setup

In [1]:
import sys
toolpath = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/src'
sys.path.append(toolpath)

%load_ext autoreload
%autoreload 2

from wtfc_utils import etl_utils as etl

import datetime
# import time
import configparser
import json
import requests
from sqlalchemy import create_engine

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import fitbit
import myfitnesspal
from nokia import NokiaAuth, NokiaApi, NokiaCredentials # Withings


In [2]:
server_dir = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/'
cfg_file = server_dir + 'config/api_params.cfg'
db_dir = server_dir + 'db/'
backups_dir = db_dir + 'backups/'
db_name = 'weightforecaster'
db_ext = '.db'
db_file_name = db_dir + db_name + db_ext

In [3]:
db_file_name

'/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/db/weightforecaster.db'

## Load DB

In [4]:
# See: https://pandas.pydata.org/pandas-docs/stable/io.html#advanced-sqlalchemy-queries
engine = create_engine('sqlite:///'+db_file_name)

In [5]:
with engine.connect() as conn, conn.begin():
    db_df = pd.read_sql_table('fitness', conn, index_col='date', parse_dates=['date'])

In [6]:
db_df.tail(5)

,weight,steps,calories,weight_imputed
date,,,,
2020-03-27,168.6,10881.0,2243.0,0
2020-03-28,168.2,5905.0,1987.0,0
2020-03-29,166.7,13504.0,1961.0,0
2020-03-30,165.4,12350.0,1859.0,0
2020-03-31,165.7,15183.0,2162.0,0


## Initialize API configs

### Nokia / Withings

See the following:
- https://github.com/orcasgit/python-nokia
- https://github.com/orcasgit/python-nokia/blob/master/nokia/__init__.py

In [96]:
parser = configparser.ConfigParser()
parser.read(cfg_file)
CLIENT_ID = parser.get('withings', 'client_id')
CLIENT_SECRET = parser.get('withings', 'client_secret')
REDIRECT_URI = parser.get('withings', 'redirect_uri')

auth = NokiaAuth(CLIENT_ID, CLIENT_SECRET, callback_uri=REDIRECT_URI)
authorize_url = auth.get_authorize_url()

# open the following in a browser, click allow, it redirects. Copy the 'code' parameter from that url.
print(authorize_url)

https://account.withings.com/oauth2_user/authorize2?response_type=code&client_id=4ce14f6c8de5fc71bed6983fa0cf85b5c43722421a9f435cf8b2b77d46c755c9&redirect_uri=https%3A%2F%2Fjamieinfinity.github.io%2Fcodegraphy%2Fblog%2F&scope=user.metrics&state=LRrDZMU526Hxcn1GLjAGMvmo6IvY4x


In [97]:
# set this to the alphanumeric string value appearing for the 'code' parameter in the url
code = 'xxx'

In [98]:
res = requests.post(url = 'https://account.withings.com/oauth2/token', 
              data = {
                  'grant_type':'authorization_code',
                  'client_id':CLIENT_ID,
                  'client_secret':CLIENT_SECRET,
                  'redirect_uri':REDIRECT_URI,
                  'code':code
              })   
token_dict = json.loads(res.content)
token_dict

b'{"access_token":"cbf5cb2220a3c212f009ada3aaa5a3f77a77d8fc","expires_in":10800,"token_type":"Bearer","scope":"user.metrics","refresh_token":"f8959d2feb5cbb4ca75011860408b618d0a20f68","userid":"2178209"}'

In [113]:
etl.persist_nokia_refresh_token(token_dict, cfg_file)

### Fitbit

See the following:

- https://python-fitbit.readthedocs.io/en/latest/

In a terminal, cd to location of python-fitbit repo (clone it from github: https://github.com/orcasgit/python-fitbit).

Then run the following:

```
python gather_keys_oauth2.py <client_key> <client_secret>
```

This script has a callback function for persisting the refresh token. Make sure the path to the api_params.cfg file is properly set (probably good to test it out…)

### MyFitnessPal

See the following:

- https://github.com/coddingtonbear/python-myfitnesspal

In a terminal, run the following command to set up authentication (locally storing your user credentials):

```
myfitnesspal store-password my_username
```

## Test etl scripting

In [7]:
db_df_updated = db_df.copy()

In [8]:
etl.get_target_date_endpoints('steps', db_df_updated)

[datetime.datetime(2020, 3, 17, 0, 0), datetime.datetime(2020, 4, 21, 0, 0)]

### Fitbit

In [9]:
db_df_updated = etl.refresh_steps(cfg_file, engine, db_df_updated)

REFRESHING STEPS...


In [10]:
db_df_updated.tail(28)

,weight,steps,calories,weight_imputed
date,,,,
2020-03-25,166.4,14243.0,1995.0,0.0
2020-03-26,167.1,11308.0,1937.0,0.0
2020-03-27,168.6,10881.0,2243.0,0.0
2020-03-28,168.2,5905.0,1987.0,0.0
2020-03-29,166.7,13504.0,1961.0,0.0
2020-03-30,165.4,12350.0,1859.0,0.0
2020-03-31,165.7,15183.0,2162.0,0.0
2020-04-01,NaN,9067.0,NaN,NaN
2020-04-02,NaN,14037.0,NaN,NaN


### MyFitnessPal

In [11]:
db_df_updated = etl.refresh_calories(engine, db_df_updated)

REFRESHING CALORIES...


In [12]:
db_df_updated.tail(28)

,weight,steps,calories,weight_imputed
date,,,,
2020-03-25,166.4,14243.0,1995.0,0.0
2020-03-26,167.1,11308.0,1937.0,0.0
2020-03-27,168.6,10881.0,2243.0,0.0
2020-03-28,168.2,5905.0,1987.0,0.0
2020-03-29,166.7,13504.0,1961.0,0.0
2020-03-30,165.4,12350.0,1859.0,0.0
2020-03-31,165.7,15183.0,2162.0,0.0
2020-04-01,NaN,9067.0,2101.0,NaN
2020-04-02,NaN,14037.0,1641.0,NaN


### Nokia / Withings

In [13]:
db_df_updated = etl.refresh_weight(cfg_file, engine, db_df_updated)

In [18]:
db_df_updated[db_df_updated.index > '2019-10-30']

,weight,steps,calories,weight_imputed
date,,,,
2019-10-31,179.200000,14476.0,580.0,NaN
2019-11-01,178.409167,24122.0,1843.0,1.0
2019-11-02,177.873564,22211.0,2252.0,1.0
2019-11-03,177.450878,20224.0,2334.0,1.0
2019-11-04,177.157086,13557.0,1786.0,1.0
...,...,...,...,...
2020-04-17,169.000000,2786.0,2426.0,NaN
2020-04-18,170.300000,12423.0,2605.0,NaN
2020-04-19,170.000000,13062.0,2558.0,NaN


## Impute weight

In [19]:
db_df_updated = etl.impute_missing_weights(engine, db_df_updated)

IMPUTING MISSING WEIGHTS...


In [20]:
db_df_updated[db_df_updated.index > '2019-10-30']

,weight,steps,calories,weight_imputed
date,,,,
2019-10-31,179.200000,14476.0,580.0,0.0
2019-11-01,178.212098,24122.0,1843.0,1.0
2019-11-02,177.594992,22211.0,2252.0,1.0
2019-11-03,177.100407,20224.0,2334.0,1.0
2019-11-04,176.751546,13557.0,1786.0,1.0
...,...,...,...,...
2020-04-17,169.000000,2786.0,2426.0,0.0
2020-04-18,170.300000,12423.0,2605.0,0.0
2020-04-19,170.000000,13062.0,2558.0,0.0


## Add rolling week-averaged columns

In [41]:
db_df_updated = etl.add_roll_avg_columns(engine, db_df_updated)

ADDING ROLLING AVG COLUMNS...


In [42]:
db_df_updated.tail(21)

,weight,steps,calories,weight_imputed,w_7day_avg,c_7day_avg,s_7day_avg,w_7day_avg_last_week,c_7day_avg_last_week,s_7day_avg_last_week,w_7day_avg_weekly_diff
date,,,,,,,,,,,
2020-04-01,168.3,9067.0,2101.0,0.0,167.142857,2035.714286,11171.142857,168.142857,2245.857143,12222.142857,-1.000000
2020-04-02,167.1,14037.0,1641.0,0.0,167.142857,1993.428571,11561.000000,167.942857,2232.714286,12637.857143,-0.800000
2020-04-03,169.1,2823.0,2292.0,0.0,167.214286,2000.428571,10409.857143,168.257143,2204.000000,11687.428571,-1.042857
2020-04-04,167.1,2422.0,2139.0,0.0,167.057143,2022.142857,9912.285714,168.142857,2213.142857,10004.142857,-1.085714
2020-04-05,166.9,3710.0,2254.0,0.0,167.085714,2064.000000,8513.142857,167.628571,2111.857143,10610.714286,-0.542857
2020-04-06,167.0,11771.0,2050.0,0.0,167.314286,2091.285714,8430.428571,167.242857,1987.714286,10263.285714,0.071429
2020-04-07,168.4,10205.0,1876.0,0.0,167.700000,2050.428571,7719.285714,166.871429,2020.571429,11910.571429,0.828571
2020-04-08,168.2,10857.0,2067.0,0.0,167.685714,2045.571429,7975.000000,167.142857,2035.714286,11171.142857,0.542857
2020-04-09,169.6,11040.0,2073.0,0.0,168.042857,2107.285714,7546.857143,167.142857,1993.428571,11561.000000,0.900000
